# Метрики на Baseline моделях

In [1]:
# install libs
!pip install datasets
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

### Проверка CUDA

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu124
True


#### Скачивание датасета

In [3]:
from datasets import load_dataset

# Load dataset from Hugging Face Hub
dataset = load_dataset("fitlemon/rag-labor-codex-dataset")

# Access train/test splits
train_data = dataset["train"]
test_data = dataset["test"]

README.md:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.32M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/620k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5264 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1316 [00:00<?, ? examples/s]

### Подготовка данных

In [4]:
corpus = test_data['chunk']
queries = test_data['question']

# corpus ids as indexes of list
corpus = dict(zip(map(str, range(len(corpus))), corpus))  # Our corpus (cid => document)
queries = dict(zip(map(str, range(len(queries))), queries))  # Our queries (qid => question)

In [5]:
# --- Step 1: Deduplicate the corpus ---

# This dictionary will help us check if a text has been seen before.
seen_texts = {}

# new_corpus will hold the deduplicated texts with new document IDs.
new_corpus = {}

# This dictionary maps old corpus IDs to the new corpus IDs.
old_to_new = {}

new_id = 0
for old_id, text in corpus.items():
    if text in seen_texts:
        # If this text is a duplicate, map the old ID to the existing new ID.
        old_to_new[old_id] = seen_texts[text]
    else:
        # Otherwise, add the text as a new entry.
        new_id_str = str(new_id)
        new_corpus[new_id_str] = text
        seen_texts[text] = new_id_str
        old_to_new[old_id] = new_id_str
        new_id += 1

print(f"Original corpus size: {len(corpus)}")
print(f"Deduplicated corpus size: {len(new_corpus)}")

# --- Step 2: Update the relevant docs mapping ---

new_relevant_docs = {}
for qid in queries.keys():
    new_relevant_docs[qid] = {old_to_new[qid]}

# --- (Optional) Use the deduplicated corpus in your evaluator ---
from sentence_transformers.evaluation import InformationRetrievalEvaluator

ir_evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=new_corpus,
    relevant_docs=new_relevant_docs,
    name="Codex-RAG-test_dedup",
)

Original corpus size: 1316
Deduplicated corpus size: 590


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Инициализация моделей

### Модель intfloat/multilingual-e5-large-instruct

In [6]:
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_mle5 = SentenceTransformer('intfloat/multilingual-e5-large-instruct', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [7]:
results_mle5 = ir_evaluator(model_mle5)

In [8]:
results_mle5

{'Codex-RAG-test_dedup_cosine_accuracy@1': 0.49012158054711247,
 'Codex-RAG-test_dedup_cosine_accuracy@3': 0.7074468085106383,
 'Codex-RAG-test_dedup_cosine_accuracy@5': 0.7689969604863222,
 'Codex-RAG-test_dedup_cosine_accuracy@10': 0.8533434650455927,
 'Codex-RAG-test_dedup_cosine_precision@1': 0.49012158054711247,
 'Codex-RAG-test_dedup_cosine_precision@3': 0.2358156028368794,
 'Codex-RAG-test_dedup_cosine_precision@5': 0.15379939209726443,
 'Codex-RAG-test_dedup_cosine_precision@10': 0.08533434650455926,
 'Codex-RAG-test_dedup_cosine_recall@1': 0.49012158054711247,
 'Codex-RAG-test_dedup_cosine_recall@3': 0.7074468085106383,
 'Codex-RAG-test_dedup_cosine_recall@5': 0.7689969604863222,
 'Codex-RAG-test_dedup_cosine_recall@10': 0.8533434650455927,
 'Codex-RAG-test_dedup_cosine_ndcg@10': 0.6687424222587454,
 'Codex-RAG-test_dedup_cosine_mrr@10': 0.6099082718193665,
 'Codex-RAG-test_dedup_cosine_map@100': 0.6149515095684647}

### Модель BAAI/bge-m3

In [9]:
model_bge = SentenceTransformer('BAAI/bge-m3', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [10]:
results_bge = ir_evaluator(model_bge)

In [11]:
results_bge

{'Codex-RAG-test_dedup_cosine_accuracy@1': 0.5349544072948328,
 'Codex-RAG-test_dedup_cosine_accuracy@3': 0.7370820668693009,
 'Codex-RAG-test_dedup_cosine_accuracy@5': 0.8161094224924013,
 'Codex-RAG-test_dedup_cosine_accuracy@10': 0.8822188449848024,
 'Codex-RAG-test_dedup_cosine_precision@1': 0.5349544072948328,
 'Codex-RAG-test_dedup_cosine_precision@3': 0.24569402228976692,
 'Codex-RAG-test_dedup_cosine_precision@5': 0.16322188449848024,
 'Codex-RAG-test_dedup_cosine_precision@10': 0.08822188449848024,
 'Codex-RAG-test_dedup_cosine_recall@1': 0.5349544072948328,
 'Codex-RAG-test_dedup_cosine_recall@3': 0.7370820668693009,
 'Codex-RAG-test_dedup_cosine_recall@5': 0.8161094224924013,
 'Codex-RAG-test_dedup_cosine_recall@10': 0.8822188449848024,
 'Codex-RAG-test_dedup_cosine_ndcg@10': 0.7074829048963801,
 'Codex-RAG-test_dedup_cosine_mrr@10': 0.6514956337144796,
 'Codex-RAG-test_dedup_cosine_map@100': 0.6559511029394972}

### Модель jinaai/jina-embeddings-v3

In [12]:
model_jina = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True, device=device)

modules.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/734k [00:00<?, ?B/s]

custom_st.py:   0%|          | 0.00/8.78k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-embeddings-v3:
- custom_st.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

In [13]:
results_jina = ir_evaluator(model_jina)

In [14]:
results_jina

{'Codex-RAG-test_dedup_cosine_accuracy@1': 0.479483282674772,
 'Codex-RAG-test_dedup_cosine_accuracy@3': 0.6770516717325228,
 'Codex-RAG-test_dedup_cosine_accuracy@5': 0.7598784194528876,
 'Codex-RAG-test_dedup_cosine_accuracy@10': 0.8343465045592705,
 'Codex-RAG-test_dedup_cosine_precision@1': 0.479483282674772,
 'Codex-RAG-test_dedup_cosine_precision@3': 0.22568389057750757,
 'Codex-RAG-test_dedup_cosine_precision@5': 0.1519756838905775,
 'Codex-RAG-test_dedup_cosine_precision@10': 0.08343465045592705,
 'Codex-RAG-test_dedup_cosine_recall@1': 0.479483282674772,
 'Codex-RAG-test_dedup_cosine_recall@3': 0.6770516717325228,
 'Codex-RAG-test_dedup_cosine_recall@5': 0.7598784194528876,
 'Codex-RAG-test_dedup_cosine_recall@10': 0.8343465045592705,
 'Codex-RAG-test_dedup_cosine_ndcg@10': 0.6540319401186032,
 'Codex-RAG-test_dedup_cosine_mrr@10': 0.5964701838182086,
 'Codex-RAG-test_dedup_cosine_map@100': 0.6028558584870086}

### Модель fitlemon/bge-m3-uz-legal-matryoshka

In [15]:
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_finetuned = SentenceTransformer('fitlemon/bge-m3-uz-legal-matryoshka', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/93.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [16]:
results_finetuned = ir_evaluator(model_finetuned)

In [17]:
results_finetuned

{'Codex-RAG-test_dedup_cosine_accuracy@1': 0.6025835866261399,
 'Codex-RAG-test_dedup_cosine_accuracy@3': 0.8252279635258358,
 'Codex-RAG-test_dedup_cosine_accuracy@5': 0.8768996960486323,
 'Codex-RAG-test_dedup_cosine_accuracy@10': 0.9300911854103343,
 'Codex-RAG-test_dedup_cosine_precision@1': 0.6025835866261399,
 'Codex-RAG-test_dedup_cosine_precision@3': 0.2750759878419453,
 'Codex-RAG-test_dedup_cosine_precision@5': 0.17537993920972644,
 'Codex-RAG-test_dedup_cosine_precision@10': 0.09300911854103341,
 'Codex-RAG-test_dedup_cosine_recall@1': 0.6025835866261399,
 'Codex-RAG-test_dedup_cosine_recall@3': 0.8252279635258358,
 'Codex-RAG-test_dedup_cosine_recall@5': 0.8768996960486323,
 'Codex-RAG-test_dedup_cosine_recall@10': 0.9300911854103343,
 'Codex-RAG-test_dedup_cosine_ndcg@10': 0.7730735786662941,
 'Codex-RAG-test_dedup_cosine_mrr@10': 0.7219463139865884,
 'Codex-RAG-test_dedup_cosine_map@100': 0.7252070263311415}

### Итоговые метрики

In [18]:
# combine all results and save in dataframe as results of models
import pandas as pd

results = pd.DataFrame({
    "mle5": results_mle5,
    "bge": results_bge,
    "finetuned_bge_4epoch_8_batch": results_finetuned,
    "jina": results_jina
})

In [19]:
results

,mle5,bge,finetuned_bge_4epoch_8_batch,jina
Codex-RAG-test_dedup_cosine_accuracy@1,0.490122,0.534954,0.602584,0.479483
Codex-RAG-test_dedup_cosine_accuracy@3,0.707447,0.737082,0.825228,0.677052
Codex-RAG-test_dedup_cosine_accuracy@5,0.768997,0.816109,0.876900,0.759878
Codex-RAG-test_dedup_cosine_accuracy@10,0.853343,0.882219,0.930091,0.834347
Codex-RAG-test_dedup_cosine_precision@1,0.490122,0.534954,0.602584,0.479483
Codex-RAG-test_dedup_cosine_precision@3,0.235816,0.245694,0.275076,0.225684
Codex-RAG-test_dedup_cosine_precision@5,0.153799,0.163222,0.175380,0.151976
Codex-RAG-test_dedup_cosine_precision@10,0.085334,0.088222,0.093009,0.083435
Codex-RAG-test_dedup_cosine_recall@1,0.490122,0.534954,0.602584,0.479483
Codex-RAG-test_dedup_cosine_recall@3,0.707447,0.737082,0.825228,0.677052


### Errors analysis

In [20]:
import torch
from sentence_transformers import SentenceTransformer, util

# (Assuming your model and dataset have been loaded as in your code)
# model_finetuned = SentenceTransformer('fitlemon/bge-m3-uz-legal-matryoshka', device=device)
# corpus and queries are dictionaries with keys "0", "1", "2", ... as in your code.

# Set the cutoff for retrieval (e.g., top 3)
top_k = 3

# Encode all corpus texts once
corpus_ids = list(new_corpus.keys())
corpus_texts = [new_corpus[cid] for cid in corpus_ids]
print("Encoding corpus texts...")
corpus_embeddings = model_finetuned.encode(corpus_texts, convert_to_tensor=True)

Encoding corpus texts...


In [21]:
corpus_embeddings.shape

torch.Size([590, 1024])

In [22]:
# To store the queries for which the relevant document is NOT retrieved
queries_with_zero_recall = []
queries_with_true_positive = []
# Loop over each query
print("Evaluating queries...")
for qid, query_text in queries.items():
    # Encode the query
    query_embedding = model_finetuned.encode(query_text, convert_to_tensor=True)

    # Compute cosine similarity between the query and all corpus documents
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]  # shape: (len(corpus),)

    # Retrieve the top_k highest scoring document indices
    top_results = torch.topk(cos_scores, k=top_k)
    top_indices = top_results.indices.tolist()

    # Convert indices back to corpus ids (which are strings like "0", "1", etc.)
    retrieved_ids = [corpus_ids[i] for i in top_indices]

    # The relevant document for this query is the one with the same key (by your mapping)
    relevant_doc = list(new_relevant_docs[qid])[0]  # since it's stored as a set

    # Check if the relevant document is in the retrieved list
    if relevant_doc not in retrieved_ids:
        queries_with_zero_recall.append((qid, retrieved_ids))
    else:
        queries_with_true_positive.append((qid, retrieved_ids))


Evaluating queries...


In [23]:
# Report the results
print(f"Found {len(queries_with_zero_recall)} queries with zero recall (at top-{top_k}).")
print("Example queries with zero recall:")
for qid, retrieved_ids in queries_with_zero_recall[:5]:
    print(f"Query ID: {qid}")
    print(f"  Query: {queries[qid]}")
    print(f"  Relevant doc (expected): {corpus[list(new_relevant_docs[qid])[0]]}")
    # (Note: If you need to keep the set intact, you can access its element via next(iter(...)))
    # retrieved docs
    print("  Retrieved docs:")
    for cid in retrieved_ids:
        print(f" CID: {cid}   - {corpus[cid]}")
    print()


Found 230 queries with zero recall (at top-3).
Example queries with zero recall:
Query ID: 1
  Query: Xodimlarning mehnatiga haq to‘lash qanday shartlarga bog‘liq?
  Relevant doc (expected): I va II guruh nogironligi bo‘lgan xodimlarning mehnatiga ish vaqtining qisqartirilgan 
davomiyligida haq to‘lash tegishli kasb va lavozimlardagi xodimlarga to‘liq ish vaqti 
davomiyligidagi miqdorda amalga oshiriladi. 
26-bob. Xodimning ish beruvchi bilan mehnatga oid aloqasi xususiyatiga va o‘z mehnat 
faoliyatini amalga oshirish joyiga bog‘liq holda mehnatni huquqiy jihatdan tartibga 
solishning o‘ziga xos xususiyatlari 
1-§. O‘rindoshlik asosida ishlovchi shaxslarning mehnatini huquqiy jihatdan tartibga 
solishning o‘ziga xos xususiyatlari
  Retrieved docs:
 CID: 179   - Mehnat sohasidagi ijtimoiy sheriklik o‘z vakillari timsolida xodimlar, ish beruvchilar, 
ularning vakillari, davlat organlari o‘rtasidagi o‘zaro munosabatlar tizimi bo‘lib, bu tizim yakka 
tartibdagi mehnatga oid munosabatlarni 

In [24]:
for qid, retrieved_ids in queries_with_zero_recall[15:20]:
    print(f"Query ID: {qid}")
    print(f"  Query: {queries[qid]}")
    print(f"  Relevant doc (expected): {corpus[qid]}")
    # (Note: If you need to keep the set intact, you can access its element via next(iter(...)))
    # retrieved docs
    print("  Retrieved docs:")
    for cid in retrieved_ids:
        print(f" CID: {cid}   - {corpus[cid]}")
    print()

Query ID: 76
  Query: Ish beruvchi mehnat shartnomasini bekor qilganda mahalliy mehnat organiga qanday xabar berishi kerak?
  Relevant doc (expected): bajarayotgan ishiga muvofiq emasligi; 
4) xodimning o‘z mehnat majburiyatlarini muntazam ravishda buzganligi. Avval mehnat 
majburiyatlarini buzganligi uchun xodim intizomiy yoki moddiy javobgarlikka tortilgan yoxud unga 
nisbatan mehnat to‘g‘risidagi qonunchilikda va mehnat haqidagi boshqa huquqiy hujjatlarda nazarda 
tutilgan ta’sir choralari qo‘llanilgan kundan e’tiboran bir yil ichida xodim tomonidan takroran 
intizomiy nojo‘ya harakat sodir etilganligi mehnat majburiyatlarini muntazam ravishda buzishdir; 
5) xodimning o‘z mehnat majburiyatlarini bir marta qo‘pol ravishda buzganligi. Xodim bilan 
tuzilgan mehnat shartnomasini bekor qilishga olib kelishi mumkin bo‘lgan mehnat majburiyatlarining 
bir marta qo‘pol ravishda buzilishlari ro‘yxatini belgilash ushbu Kodeksning 162-moddasiga muvofiq 
amalga oshiriladi;  
6) ushbu Kodeksda va

In [ ]:
# print relevant docs
for qid, retrieved_ids in queries_with_true_positive[:5]:
    print(f"Query ID: {qid}")
    print(f"  Query: {queries[qid]}")
    print(f"  Relevant doc (expected): {corpus[qid]}")
    for cid in retrieved_ids:
        print(f" CID: {cid}   - {corpus[cid]}")
    print()

Query ID: 0
  Query: Komissiyalar vakolatlarining muddati qanday belgilanadi?
  Relevant doc (expected): Ijtimoiy-mehnat masalalari bo‘yicha komissiyalar doimiy faoliyat ko‘rsatuvchi organlardir. 
Komissiyalar vakolatlarining muddati tegishli jamoa kelishuvini yoki jamoa shartnomasini 
tayyorlash va uning amal qilish muddatiga belgilanadi. 
Taraflar tomonidan yangi jamoa kelishuvini, jamoa shartnomasini tayyorlash to‘g‘risida 
qaror qabul qilingan taqdirda ular ushbu Kodeksning 48 — 51-moddalarida nazarda tutilgan tartibda 
yangidan ijtimoiy-mehnat masalalari bo‘yicha tegishli komissiyalarni tuzadi. Bunda, agar ijtimoiy -
mehnat masalalari bo‘yicha komissiya ishining yakunlariga ko‘ra tegishli taraf avvalgi komissiya 
tarkibida bo‘lgan shaxslarning faoliyatini qoniqarli deb topgan bo‘lsa, tegishli tarafning qarori bilan 
komissiya tarkibiga ushbu shaxslar ham kiritilishi mumkin.
 CID: 0   - Ijtimoiy-mehnat masalalari bo‘yicha komissiyalar doimiy faoliyat ko‘rsatuvchi organlardir. 
Komi

In [25]:
# Calculate recall@3 as the ratio of queries where the relevant doc was retrieved in the top 3
total_queries = len(queries)
true_positive_queries = len(queries_with_true_positive)
recall_at_3 = true_positive_queries / total_queries

print(f"Recall@3: {recall_at_3:.5f}")

Recall@3: 0.82523
